## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [ ]:
import os
from utils import setup_environment, load_data, setup_logging

from retriever import retriever
from planner import planner
from dreamer import dreamer
from grounder import grounder
from executor import executor

from schemas import RetrieverOut, PlannerOut, DreamerOut

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне
# db = db.sample(1000, random_state=42)               # Ограничение по числу строк - уменьшаем контексты

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [ ]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # "model": "deepseek/deepseek-chat-v3.1",
        "temperature": 0.8,
        "reasoning_effort": None
    },
    # параметры аналитика-выдумщика
    dreamer_params={
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # "model": "deepseek/deepseek-chat-v3.1",
        "temperature": 1,
        "reasoning_effort": "high"
    },
    # параметры планировщика
    planner_params={
        "model": "openai/gpt-oss-20b:free",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # "model": "deepseek/deepseek-chat-v3.1",
        "temperature": 0.2
    }
)

### Logger

Настройка логирования и директорий сохранения файлов

In [ ]:
run_dir = setup_logging(mode="DEBUG", log_dir="logs")
# run_dir = r"C:\Users\kateu\Documents\IvRAG\logs\run_2025-11-14_15-33-01"

retriever_out_path = os.path.join(run_dir, "retrieved.json")
dreamer_out_path = os.path.join(run_dir, "dreamed.json")
planner_out_path = os.path.join(run_dir, "planned.json")

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [ ]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
# user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
# user_query = "Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть, среди жителей Москвы"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

retriever_out = retriever(user_query, PPL_cfg)
retriever_out.save(retriever_out_path)

Можно посмотреть reasoning, если модель его поддерживает и он был указан в параметрах

In [ ]:
# print(retriever_out.reasoning)

In [ ]:
retriever_out = RetrieverOut.load(retriever_out_path)
print(retriever_out)

### Dreamer

Строит план анализа без привязки к функционалу

In [ ]:
dreamer_out = dreamer(user_query, PPL_cfg)
dreamer_out.save(dreamer_out_path)

In [ ]:
dreamer_out = DreamerOut.load(dreamer_out_path)
print(dreamer_out)

Можно отдельно посмотреть **анализ** и **reasoning**, раскомментировав соответствующую строку

In [ ]:
# print(dreamer_out.analysis)
# print(dreamer_out.reasoning)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, dreamer_out, PPL_cfg)
planner_out.save(planner_out_path)

In [ ]:
planner_out = PlannerOut.load(planner_out_path)
print(planner_out)

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [ ]:
grounder_out = grounder(planner_out)

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [ ]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

Далее можно извлекать созданные таблицы из `ctx`

In [ ]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["intersection_counts"]